In [18]:
#resolve Dependencies
import pandas as pd
import os
import pymongo
from pymongo import MongoClient

In [19]:
free_reduced_src_path = os.path.join('..','static', 'data', 'processed', 'free_reduced_lunch_ny_county.csv')

ny_population_race_breakdown_src = os.path.join('..','static', 'data', 'processed', 'population_race_breakdown.csv')

covid_ny_county_src_path = os.path.join('..','static', 'data', 'processed', 'covid_ny_county.csv')

median_income_src_path = os.path.join('..','static', 'data', 'processed', 'median_income.csv')




In [20]:
#read data files into frames
free_reduced_df = pd.read_csv(free_reduced_src_path)

population_race_breakdown_df = pd.read_csv(ny_population_race_breakdown_src)

covid_results_df = pd.read_csv(covid_ny_county_src_path)
    
median_income_df = pd.read_csv(median_income_src_path)


covid_results_df

,id,County,State,Date,Population,Deaths,Cases,% of Population,% of Deaths,% of Cases
0,0500000US36NYS,Statewide Unallocated,NY,5/22/2020,0,90,69,0.0,0.0,0.0
1,0500000US36NYC,New York City Unallocated,NY,5/22/2020,0,310,0,0.0,1.0,0.0
2,0500000US36001,Albany,NY,5/22/2020,305506,94,1756,2.0,0.0,0.0
3,0500000US36003,Allegany,NY,5/22/2020,46091,2,44,0.0,0.0,0.0
4,0500000US36005,Bronx,NY,5/22/2020,1418207,4201,43766,7.0,15.0,12.0
...,...,...,...,...,...,...,...,...,...,...
60,0500000US36117,Wayne,NY,5/22/2020,89918,2,104,0.0,0.0,0.0
61,0500000US36119,Westchester,NY,5/22/2020,967506,1319,32767,5.0,5.0,9.0
62,0500000US36121,Wyoming,NY,5/22/2020,39859,5,79,0.0,0.0,0.0
63,0500000US36123,Yates,NY,5/22/2020,24913,6,34,0.0,0.0,0.0


In [21]:
# build a single master dataframe using a select set of columns from each
# dataframe
# #select and rename columns
population_raw_df = population_race_breakdown_df[['County','overall','white','black','native_american','asian','pacific_islander','other', 'hispanic_latino']].add_suffix('_population')
population_df = population_raw_df.rename(columns={'County_population': 'County'})


In [22]:
medianincome_df = median_income_df[['County','asian','black','hispanic','native_american','other','pacific_islander','white']].add_suffix('_median_income')
medianincome_df

,County_median_income,asian_median_income,black_median_income,hispanic_median_income,native_american_median_income,other_median_income,pacific_islander_median_income,white_median_income
0,Hamilton,NaN,NaN,NaN,NaN,NaN,NaN,115546
1,Schoharie,96429.0,63750.0,63438.0,NaN,99167.0,NaN,107570
2,Onondaga,46613.0,31623.0,36629.0,39747.0,73268.0,NaN,130387
3,Clinton,40729.0,50208.0,57019.0,NaN,88564.0,NaN,111166
4,Seneca,NaN,36094.0,35172.0,NaN,60636.0,NaN,111715
...,...,...,...,...,...,...,...,...
57,Greene,98462.0,64026.0,41641.0,NaN,NaN,NaN,106510
58,Allegany,NaN,41406.0,19141.0,34239.0,NaN,NaN,94491
59,Cattaraugus,66316.0,NaN,34375.0,28625.0,65391.0,NaN,96416
60,Schenectady,66726.0,31813.0,40604.0,54135.0,82169.0,NaN,138807


In [24]:


df1 = pd.merge(population_df,medianincome_df , how="left", left_on="County", right_on="County_median_income")

In [25]:
#select and rename columns
covid_ny_raw_df = covid_results_df[['County','Date','Deaths','Cases','% of Population','% of Deaths','% of Cases']].add_prefix('covid_')
covid_ny_df = covid_ny_raw_df.rename(columns={'covid_County':'identified_county','covid_Date': 'covid_date', 'covid_Deaths':'covid_deaths','covid_Cases':'covid_cases', 'covid_% of Deaths': 'covid_death_%','covid_% of Cases': 'covid_cases_%','covid_% of Population': 'covid%ofpopulation' })

In [26]:
# perform merge for master analysis collection
df2 = pd.merge(df1, covid_ny_df, how="left", left_on="County", right_on="identified_county")

In [27]:
# connect to mongodb
client = MongoClient('mongodb://localhost:27017')

In [28]:
#set db connection
db = client['nycares_db']

In [29]:
#create dictionaries to populate mongo collects

free_as_dict = free_reduced_df.to_dict('records')

population_as_dict = population_race_breakdown_df.to_dict('records')

covid_results_dict = covid_results_df.to_dict('records')

median_income_dict = median_income_df.to_dict('records')

covid_analysis_dict = df2.to_dict('records')


In [30]:
#set reference to collection
frlp_collection = db['free_and_reduced_lunch']
frlp_collection.delete_many({})

population_race_collection = db['population_race_breakdown']
population_race_collection.delete_many({})

covid_results_collection = db['covid_results_by_county']
covid_results_collection.delete_many({})

median_income_collection = db['median_income']
median_income_collection.delete_many({})

covid_master_collection = db['covid_master_analysis']
covid_master_collection.delete_many({})

In [31]:
# insert documents to collections
for record in free_as_dict:
     frlp_collection.insert_one(record)
    

for record in population_as_dict:
    population_race_collection.insert_one(record)
    

for record in covid_results_dict:
    covid_results_collection.insert_one(record)

    
for record in median_income_dict:
    median_income_collection.insert_one(record)
    
for record in covid_analysis_dict:
    covid_master_collection.insert_one(record)

In [32]:
df2.to_csv(os.path.join('..','static','data','processed','covid_master_analysis.csv'))